In [ ]:
import os
import pandas as pd
import numpy as np
import scipy.sparse as sp
import json
import random
import regex
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
import time
"""
This script trains a MLP classifier to predict the binding affinity of a 30 bases sequence w.r.t. the obtained priming window.

The training data is obtained from the upstream 30 bases of the alignments of R1 in our paired-end read alignment results from the genome. The training data has the same orientation as its read2, so, it should contain a polyA site.
As reads might come from polyA tails, which are not included in the genome, we would expect that some of the input data come from intergenic regions and do not correspond to a polyA site. This is saying our labels are noisy, so we would not expect to obtain perfect accuracy.

We train a MLP classifier by reading the training data from each SRR and call `partial_fit` to update the model. 

"""

parser = json.load(open("params.json", "r"))
parent_dir = parser["parent_dir"]
PE_sheet = parser["PE_sheet"]
random_seed = parser["random_seed"]
outdir_parent = parser["outdir"]
snr_len = parser["snr_len"]
snr_mismatch = parser["snr_mismatch"]

print(parser)


In [ ]:

encoder = OneHotEncoder(categories=[['A', 'C', 'G', 'T', 'N']] * 30, handle_unknown='ignore')

parent_dir = parent_dir
# 1----- Get the PE datasets spreadsheet
PE_sheet = pd.read_csv(PE_sheet)

# 2----- loop through GSE(s), combine all tlen from its SRR
# check if we have all datasets processed
missing_files = []

for (GSE, group_gse_lst) in PE_sheet.groupby('GSE'):
    SRR_lst = group_gse_lst['SRR']
    print(GSE)
    for SRR in SRR_lst:
        polya_path = os.path.join(
            parent_dir, "process_data", "frag_len_dist", GSE, SRR, "priming_site_seqs", "polya_seq.txt")

        if os.path.exists(polya_path):
            check_file = os.path.getsize(polya_path)
            if (check_file == 0):
                missing_files.append(f"{GSE}-{SRR}")
                error_occur = True
        else:
            missing_files.append(f"{GSE}-{SRR}")
            error_occur = True

if missing_files:
    raise ValueError(f"Please re-run the previous step, the output of following dataset(s) is either missing or empty: {missing_files}")


In [ ]:

## first run are interruped.
hidden_layer_sizes_list = [50,100]
max_iter_list = [100,200,500]


for hidden_layer_sizes in hidden_layer_sizes_list:
    for max_iter in max_iter_list:
        start_time = time.time()
        outdir = os.path.join(outdir_parent,f'/seed{random_seed}_layer_{hidden_layer_sizes}_iter_{max_iter}') 
        os.makedirs(outdir, exist_ok=True)

        # ------------------- Train the MLP model -------------------
        polya_mlp = MLPClassifier(hidden_layer_sizes=(hidden_layer_sizes,), max_iter=max_iter, alpha=1e-4,
                            solver='adam', verbose=10, random_state=random_seed, shuffle=True,
                            learning_rate_init=0.001)

        held_out_polya = sp.csr_matrix((0, 150))
        held_out_polya_bg = sp.csr_matrix((0, 150)) 
        random.seed(random_seed)
        train_list = PE_sheet.loc[PE_sheet['GSE'].isin(random.sample(PE_sheet['GSE'].unique().tolist(), 8))]
        num_srr = {GSE: group_gse_lst.shape[0] for (GSE, group_gse_lst) in train_list.groupby('GSE')}
        # initialize the sparse matrix 
        training_batch = sp.csr_matrix((0, 150))

        training_data_acc_list_during_train = []

        # we group by GSE, then loop through SRRs to get the polya seq
        for i, (GSE, group_gse_lst) in enumerate(train_list.groupby('GSE')):
            SRR_lst = group_gse_lst['SRR']
            num_held_out = 1000//len(SRR_lst)
            
            for SRR in SRR_lst:
                # The 30 bases are defined according to R2. So, they should always be polyA

                polya_path = os.path.join(
                    parent_dir, "process_data", "frag_len_dist", GSE, SRR, "priming_site_seqs", "polya_seq.txt")
                bg_path = os.path.join(
                    parent_dir, "process_data", "frag_len_dist", GSE, SRR, "priming_site_seqs", "polya_bg_seq.txt")

                # get polya and bg seq
                # we want to make sure the polya seq has a polyA six mer with at most one mismatch
                polya_batch = encoder.fit_transform([list(x.strip()) for x in open(polya_path).readlines() if len(x.strip()) == 30 and regex.search("A(" + 'A' * (snr_len-2)+ "){s<=" + str(snr_mismatch) +"}A", x.strip())])

                # polya_batch = encoder.fit_transform([list(x.strip()) for x in open(polya_path).readlines() if len(x.strip()) == 30])
                
                # if we do not have enough data, then skip this SRR
                if polya_batch.shape[0] < num_held_out * 2:
                    print("Not enough training examples for", GSE, SRR)
                    continue
                
                bg_batch = encoder.fit_transform([list(x.strip()) for x in open(bg_path).readlines() if len(x.strip()) == 30])
                
                # if we do not have enough data, then skip this SRR
                if bg_batch.shape[0] < num_held_out * 2:
                    print("Not enough background examples for", GSE, SRR)
                    continue
                
                # num_held_out = min(math.ceil(polya_batch.shape[0] * 0.1), 20)
                num_train_fg = polya_batch.shape[0]
                num_train_bg = min(num_train_fg, bg_batch.shape[0])

                polya_batch = polya_batch[np.random.randint(0, polya_batch.shape[0], size = num_train_fg), : ]
                bg_batch = bg_batch[np.random.randint(0, bg_batch.shape[0], size = num_train_bg), : ]
                
                fg_shuf_id = np.arange((num_train_fg+num_train_bg-num_held_out*2))
                np.random.shuffle(fg_shuf_id)

                # build training data
                train_polya = sp.vstack([
                    polya_batch[num_held_out:],
                    bg_batch[num_held_out:]
                ])[fg_shuf_id,:]
                
                del polya_batch
                del bg_batch
                
                label_polya = np.hstack([
                    np.ones(num_train_fg - num_held_out), 
                    np.zeros(num_train_bg - num_held_out)
                ])[fg_shuf_id]
                
                # fit the model using the data from this SRR
                polya_mlp.partial_fit(
                    train_polya,
                    label_polya,
                    classes=[0,1]
                )
                acc = accuracy_score(label_polya, 
                    polya_mlp.predict(train_polya))
                training_data_acc_list_during_train.append(acc)
                # get the accuracy on the held out data
                print("Accuracy:", acc)


        print("The mean accuracy on the training datasets is:", sum(training_data_acc_list_during_train)/len(training_data_acc_list_during_train))

        # ------------------- Test the model(partI)-------------------
        # Test the model on the training sets and held out sets for each srr
        random.seed(random_seed)
        train_list = PE_sheet.loc[PE_sheet['GSE'].isin(random.sample(PE_sheet['GSE'].unique().tolist(), 8))]
        num_srr = {GSE: group_gse_lst.shape[0] for (GSE, group_gse_lst) in train_list.groupby('GSE')}
        # initialize the sparse matrix 
        training_batch = sp.csr_matrix((0, 150))
        held_out_polya = sp.csr_matrix((0, 150))
        held_out_polya_bg = sp.csr_matrix((0, 150)) 

        training_data_acc_list = []
        held_out_data_acc_list = []

        # we group by GSE, then loop through SRRs to get the polya seq
        for i, (GSE, group_gse_lst) in enumerate(train_list.groupby('GSE')):
            SRR_lst = group_gse_lst['SRR']
            num_held_out = 1000//len(SRR_lst)
            # inital the matrix/list for each GSE
            train_polya_per_gse = sp.csr_matrix((0, 150))
            train_label_per_gse = np.array([])

            held_out_polya_per_gse = sp.csr_matrix((0, 150))
            held_out_label_per_gse = np.array([])

            for SRR in SRR_lst:
                # The 30 bases are defined according to R2. So, they should always be polyA
                polya_path = os.path.join(
                    parent_dir, "process_data", "frag_len_dist", GSE, SRR, "priming_site_seqs", "polya_seq.txt")
                bg_path = os.path.join(
                    parent_dir, "process_data", "frag_len_dist", GSE, SRR, "priming_site_seqs", "polya_bg_seq.txt")

                # get polya and bg seq
                # we want to make sure the polya seq has a polyA six mer with at most one mismatch
                polya_batch = encoder.fit_transform([list(x.strip()) for x in open(polya_path).readlines() if len(x.strip()) == 30 and regex.search("A(" + 'A' * (snr_len-2)+ "){s<=" + str(snr_mismatch) +"}A", x.strip())])
                
                # polya_batch = encoder.fit_transform([list(x.strip()) for x in open(polya_path).readlines() if len(x.strip()) == 30])
                
                # if we do not have enough data, then skip this SRR
                if polya_batch.shape[0] < num_held_out * 2:
                    print("Not enough training examples for", GSE, SRR)
                    continue
                
                bg_batch = encoder.fit_transform([list(x.strip()) for x in open(bg_path).readlines() if len(x.strip()) == 30])
                
                # if we do not have enough data, then skip this SRR
                if bg_batch.shape[0] < num_held_out * 2:
                    print("Not enough background examples for", GSE, SRR)
                    continue
                
                # num_held_out = min(math.ceil(polya_batch.shape[0] * 0.1), 20)
                num_train_fg = polya_batch.shape[0]
                num_train_bg = min(num_train_fg, bg_batch.shape[0])

                polya_batch = polya_batch[np.random.randint(0, polya_batch.shape[0], size = num_train_fg), : ]
                bg_batch = bg_batch[np.random.randint(0, bg_batch.shape[0], size = num_train_bg), : ]

                # append held out data
                polya_held_out = polya_batch[:num_held_out,]
                bg_held_out = bg_batch[:num_held_out,]
                
                fg_shuf_id = np.arange((num_train_fg+num_train_bg-num_held_out*2))
                np.random.shuffle(fg_shuf_id)

                # build training data
                train_polya = sp.vstack([
                    polya_batch[num_held_out:],
                    bg_batch[num_held_out:]
                ])[fg_shuf_id,:]
                
                del polya_batch
                del bg_batch
                
                label_polya = np.hstack([
                    np.ones(num_train_fg - num_held_out), 
                    np.zeros(num_train_bg - num_held_out)
                ])[fg_shuf_id]

                # group the train data by gse
                train_polya_per_gse = sp.vstack([train_polya_per_gse, train_polya])
                train_label_per_gse = np.hstack([train_label_per_gse, label_polya])

                #group the held out data by gse
                held_out_label_per_gse = np.hstack([held_out_label_per_gse, np.hstack([np.ones(num_held_out), np.zeros(num_held_out)])])
                held_out_polya_per_gse = sp.vstack([held_out_polya_per_gse, sp.vstack([polya_held_out, bg_held_out])])

            # accuracy for classification on the training data
            acc_train = accuracy_score(train_label_per_gse, 
                polya_mlp.predict(train_polya_per_gse))
            training_data_acc_list.append(acc_train)
            print("Accuracy on training data:", acc_train)

            # accuracy for classification on the held out data
            acc_held_out = accuracy_score(held_out_label_per_gse, 
                polya_mlp.predict(held_out_polya_per_gse))
            held_out_data_acc_list.append(acc_held_out)
            print("Accuracy on held out data:", acc_held_out)

        print("The mean accuracy on the training datasets is:", sum(training_data_acc_list)/len(training_data_acc_list))
        print("The mean accuracy on the held out datasets is:", sum(held_out_data_acc_list)/len(held_out_data_acc_list))

        # --------- Test the model (partII)-----------------
        # we also want to test the accuracy on test datasets
        # we read in the test data
        random.seed(random_seed)
        test_list = PE_sheet.loc[~PE_sheet['GSE'].isin(random.sample(PE_sheet['GSE'].unique().tolist(), 8))]
        test_acc_list = []
        # we group by GSE, then loop through SRRs to get the polya seq
        for i, (GSE, group_gse_lst) in enumerate(test_list.groupby('GSE')):
            SRR_lst = group_gse_lst['SRR']

            test_polya_per_gse = sp.csr_matrix((0, 150))
            test_label_per_gse = np.array([])
            
            for SRR in SRR_lst:
                # The 30 bases are defined according to R2. So, they should always be polyA

                polya_path = os.path.join(
                    parent_dir, "process_data", "frag_len_dist", GSE, SRR, "priming_site_seqs", "polya_seq.txt")
                bg_path = os.path.join(
                    parent_dir, "process_data", "frag_len_dist", GSE, SRR, "priming_site_seqs", "polya_bg_seq.txt")

                # get polya and bg seq
                # we want to make sure the polya seq has a polyA six mer with at most one mismatch
                polya_batch = encoder.fit_transform([list(x.strip()) for x in open(polya_path).readlines() if len(x.strip()) == 30 and regex.search("A(" + 'A' * (snr_len-2)+ "){s<=" + str(snr_mismatch) +"}A", x.strip())])
                # polya_batch = encoder.fit_transform([list(x.strip()) for x in open(polya_path).readlines() if len(x.strip()) == 30])
                
                # if we do not have enough data, then skip this SRR
                if polya_batch.shape[0] < num_held_out * 2:
                    print("Not enough training examples for", GSE, SRR)
                    continue
                
                bg_batch = encoder.fit_transform([list(x.strip()) for x in open(bg_path).readlines() if len(x.strip()) == 30])
                
                # if we do not have enough data, then skip this SRR
                if bg_batch.shape[0] < num_held_out * 2:
                    print("Not enough background examples for", GSE, SRR)
                    continue
                
                # build training data
                test_polya = sp.vstack([
                    polya_batch[random.sample(range(polya_batch.shape[0]), min(polya_batch.shape[0], bg_batch.shape[0])), : ],
                    bg_batch[random.sample(range(bg_batch.shape[0]), min(polya_batch.shape[0], bg_batch.shape[0])), : ]
                ])
                label_polya = np.hstack([
                    np.ones(min(polya_batch.shape[0], bg_batch.shape[0])), 
                    np.zeros(min(polya_batch.shape[0], bg_batch.shape[0]))
                ])

                del polya_batch
                del bg_batch

                # group the test data by gse
                test_polya_per_gse = sp.vstack([test_polya_per_gse, test_polya])
                test_label_per_gse = np.hstack([test_label_per_gse, label_polya])
                
                
            # get the accuracy on the held out data
            acc_scores = accuracy_score(
                test_label_per_gse, 
                polya_mlp.predict(test_polya_per_gse))
            test_acc_list.append(acc_scores)
            print("Accuracy:", acc_scores)


        print("The mean accuracy on the test datasets is:", sum(test_acc_list)/len(test_acc_list))

        # outdir = '/fs/nexus-projects/sc_frag_len/nextflow/workflow_output/models/mlp_model'
        accuracy_file_path = os.path.join(outdir, "accuracy_3_lists.pkl")
        joblib.dump([training_data_acc_list, held_out_data_acc_list, test_acc_list], accuracy_file_path)

        [training_data_acc_list, held_out_data_acc_list, test_acc_list]= joblib.load(accuracy_file_path, "rb")
        # accuracy log
        acc_dict = {
            "training_data_acc_list": training_data_acc_list,
            "held_out_data_acc_list": held_out_data_acc_list,
            "test_acc_list": test_acc_list
        }
        with open(os.path.join(outdir, 'acc_list_log.txt'),'w') as file:
            file.write(f"The time used for training the model is: {time.time() - start_time} seconds\n")
            for l_name, acc_list in acc_dict.items():
                file.write( '---------'+ "\n")
                file.write(l_name + "\n")
                for acc_num in acc_list:
                        file.write(str(acc_num)+"\n")
        file.close()
        
        #------------------- Save the model -------------------
        model_pkl_path = os.path.join(outdir, f'mlp.pkl')
        joblib.dump(polya_mlp, model_pkl_path)

        # Combine the lists into a single dataset with corresponding labels
        data = training_data_acc_list + held_out_data_acc_list + test_acc_list
        labels = ['Training'] * len(training_data_acc_list) + ['Hold out'] * len(held_out_data_acc_list) + ['Test'] * len(test_acc_list)
        print(f"training_data_acc_list: {training_data_acc_list}")
        print(f"held_out_data_acc_list: {held_out_data_acc_list}")
        print(f"test_acc_list: {test_acc_list}")
        # Create a DataFrame for easier plotting with Seaborn
        df = pd.DataFrame({'Data': data, 'Datasets': labels})

        # Create the violin plot
        sns.violinplot(x='Datasets', y='Data', data=df)
        plt.title("Distribution of prediction accuracy of MLP")
        plt.ylabel('Accuracy')
        plt.savefig(os.path.join(outdir, f"violin_plot_per_gse.png"),dpi = 300)
        plt.show()


        

